In [2]:
import time
import threading
import multiprocessing
import itertools
import os
import logging
import random
import string
import requests
from functools import partial
from multiprocessing import Queue
from multiprocessing.pool import ThreadPool

# Definitions

In [3]:
# Process
class Process:
    pid = 0
    prio = 10
    files = []
    sockets = []
    addresses = []
    state = states.READY
    
    def __init__():
        pass
    
    def process_signal(signal):
        if signal == 'SIGSTOP':
            self._stop()
        elif signal == 'SIGCONT':
            self._cont()
        elif signal == 'SIGKILL':
            self._kill()
        ...
            
    def _stop():
        pass
    
    def _cont():
        pass
        
    def _kill():
        pass

NameError: name 'states' is not defined

In [ ]:
# scheduler
for process in processes:
    process.process_signal(SIGCONT)
    # ...
    process.process_signal(SIGSTOP)   

In [ ]:
# Process = threads+
Scheduling is repeated on pthreads level

In [ ]:
class Thread:
    tid = 0
    prio = 10
    state = states.READY
    parent = None
    
    def __init__():
        pass
    
    # ...

In [ ]:
# scheduler
for thread in threads:
    thread.process_signal(SIGCONT)
    # ...
    thread.process_signal(SIGSTOP)   

In [ ]:
# 2 kinds of tasks:
- background execution: 
    - daemon process: OS services
    - multithreaded process: notifications, monitor, watcher, etc
- parallel execution: speedup

In [5]:
Parallel execution can be done in 2 ways:
    - threads
    - processes
    
Which one is better?

# Threads

In [4]:
class timer():
    def __init__(self, message):
        self.message = message

    def __enter__(self):
        self.start = time.time()
        return None

    def __exit__(self, type, value, traceback):
        elapsed_time = (time.time() - self.start)
        print(self.message.format(elapsed_time))

TIME_TO_SLEEP = 1
        
def long_running_task(n=TIME_TO_SLEEP):
    #print(threading.current_thread())
    time.sleep(n)

with timer('Elapsed: {}s'):
#     long_running_task()
    long_running_task()


Elapsed: 1.000542402267456s


In [5]:
with timer('Elapsed: {}s'):
    t1 = threading.Thread(target=long_running_task, args=(TIME_TO_SLEEP/2,))
    t2 = threading.Thread(target=long_running_task, args=(TIME_TO_SLEEP/2,))
    t1.start()
    t2.start()
    #print(threading.current_thread())
    t1.join() # be carefull
    t2.join() 
    
print('next step')
# work with date

Elapsed: 0.500849723815918s
next step


In [6]:
def run_threads(func, data, workers):
    threads = [
        threading.Thread(target=func, args=(data / workers, ))
        for _ in range(workers)
    ]
    for t in threads:
        t.start()
    for t in threads:
        t.join()

In [7]:
workers = 10
DATA_SIZE = 1

with timer('Elapsed: {}s'):
    run_threads(long_running_task, DATA_SIZE, workers)

Elapsed: 0.1357557773590088s


In [100]:
def generate_task_queue(total_tasks, queue_size):
    task_queue = []
    remainder = total_tasks
    while remainder > 0:
        value = min(remainder, random.randint(1, total_tasks//queue_size))
        remainder -= value
        task_queue.append(value)
    return task_queue

assert sum(generate_task_queue(1000, 20)) == 1000
assert sum(generate_task_queue(1000, 1)) == 1000
assert sum(generate_task_queue(1000, 1000)) == 1000
assert sum(generate_task_queue(1000, 42)) == 1000

In [8]:
input_data = [DATA_SIZE / workers for _ in range(workers)]
print(input_data)

with timer('Elapsed: {}s'):
    with ThreadPool(workers) as pool:
        
        pool.map(long_running_task, input_data)
#         for chunk in input_data:
#             long_running_task(chunk)

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.347205732259127, 0.08582011443225313, 0.5360061306118391, 0.030968022696780784]
Elapsed: 0.5447676181793213s


<img src="https://www.nginx.com/wp-content/uploads/2016/07/thread-pools-worker-process-event-cycle.png">

# Real world task

In [18]:
def fetch_pic(num_pic):
# def fetch_pic(num_pic, path):
    url = 'https://picsum.photos/400/600'
    path = './pics'
    for _ in range(num_pic):
        random_name = ''.join(random.choices(string.ascii_letters + string.digits, k=5))
        response = requests.get(url)
        if response.status_code == 200:
            with open(f'{path}/{random_name}.jpg', 'wb') as f:
                f.write(response.content)
                #print(f"Fetched pic [{os.getpid()}]: {f.name}")

In [130]:
workers = 80
DATA_SIZE = 100

with timer('Elapsed: {}s'):
    with ThreadPool(workers) as pool:
        input_data = [DATA_SIZE // workers for _ in range(workers)]
        # input_data = [(DATA_SIZE // workers, './pics') for _ in range(workers)]
        pool.map(fetch_pic, input_data)

Elapsed: 1.6135263442993164s


# IO vs CPU bound tasks

In [138]:
DATA_SIZE = 1_000_000
        


        
lst = []
with timer('Elapsed: {}s'):
#     fill_data(DATA_SIZE, lst)
    fill_data(DATA_SIZE)
    
print(len(lst), lst[:100])

Elapsed: 0.7369067668914795s
1000000 [39, 62, 1, 83, 85, 88, 22, 94, 63, 86, 40, 15, 73, 90, 82, 37, 39, 46, 43, 96, 57, 14, 50, 43, 61, 27, 54, 80, 43, 90, 88, 99, 54, 2, 14, 14, 95, 54, 12, 57, 37, 20, 95, 56, 38, 70, 4, 71, 85, 23, 79, 100, 90, 96, 96, 82, 78, 71, 84, 22, 17, 20, 68, 92, 28, 32, 49, 91, 20, 74, 52, 13, 68, 28, 33, 34, 69, 53, 45, 11, 78, 36, 98, 93, 98, 56, 65, 10, 43, 94, 83, 10, 34, 48, 91, 29, 87, 54, 60, 35]


In [151]:
lst = []

def fill_data(n):
    # print(threading.current_thread())
#     lst = []
    while n > 0:
        n -= 1
        lst.append(random.randint(1, 100))
    return lst
        
with timer('Elapsed: {}s'):
#     t1 = threading.Thread(target=fill_data, args=(DATA_SIZE // 2, lst))
#     t2 = threading.Thread(target=fill_data, args=(DATA_SIZE // 2, lst))
    t1 = threading.Thread(target=fill_data, args=(DATA_SIZE // 2,))
    t2 = threading.Thread(target=fill_data, args=(DATA_SIZE // 2 - 1,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    
print(len(lst), lst[:100])

Elapsed: 1.0093750953674316s
0 []


In [142]:
lst = []
workers = 4
with timer('Elapsed: {}s'):
    with ThreadPool(workers) as pool:
        input_data = [DATA_SIZE // workers for _ in range(workers)]
#         pool.map(partial(fill_data, lst=lst), input_data)
        pool.map(fill_data, input_data)
        
print(len(lst), lst[:100])

Elapsed: 1.030395269393921s
1000000 [58, 51, 35, 76, 16, 51, 74, 25, 39, 32, 55, 92, 11, 88, 46, 57, 10, 76, 32, 66, 100, 45, 84, 92, 72, 97, 37, 49, 69, 1, 89, 71, 39, 86, 44, 32, 21, 64, 71, 74, 94, 27, 99, 90, 93, 19, 61, 4, 41, 37, 96, 79, 75, 97, 3, 74, 42, 24, 57, 5, 81, 55, 95, 14, 75, 8, 89, 40, 45, 38, 19, 75, 70, 77, 69, 34, 7, 12, 85, 77, 5, 8, 38, 69, 1, 23, 11, 38, 21, 95, 27, 97, 39, 32, 80, 61, 37, 2, 51, 99]


In [154]:
lst = []
workers = 4
with timer('Elapsed: {}s'):
    with multiprocessing.Pool(workers) as pool:
        input_data = [DATA_SIZE // workers for _ in range(workers)]
#         pool.map(partial(fill_data, lst=lst), input_data)
        lst = pool.map(fill_data, input_data)
        
print(len(lst), len(lst[0]), lst[0][:100])

Elapsed: 0.41368770599365234s
4 250000 [4, 58, 16, 4, 9, 61, 48, 94, 81, 2, 17, 27, 76, 53, 59, 8, 20, 28, 23, 14, 54, 6, 45, 85, 20, 12, 74, 9, 17, 60, 99, 15, 87, 74, 21, 15, 10, 27, 11, 59, 20, 94, 92, 70, 93, 45, 87, 14, 86, 47, 16, 78, 3, 84, 17, 50, 96, 93, 45, 72, 80, 92, 57, 6, 8, 76, 32, 15, 26, 13, 67, 5, 1, 97, 25, 66, 69, 1, 96, 15, 45, 69, 62, 99, 66, 58, 80, 95, 38, 24, 100, 25, 38, 18, 74, 68, 34, 56, 5, 63]


In [ ]:
lst = []
workers = 4
with timer('Elapsed: {}s'):
    with multiprocessing.Pool(workers) as pool:
        input_data = [DATA_SIZE // workers for _ in range(workers)]
        result = pool.map(fill_data, input_data)
        
print(len(result), lst[:100])

In [103]:
def factorize_naive(n):
    """ A naive factorization method. Take integer 'n', return list of
        factors.
    """
    if n < 2:
        return []
    factors = []
    p = 2

    while True:
        if n == 1:
            return factors

        r = n % p
        if r == 0:
            factors.append(p)
            n = n / p
        elif p * p >= n:
            factors.append(n)
            return factors
        elif p > 2:
            # Advance in steps of 2 over odd numbers
            p += 2
        else:
            # If p == 2, get to 3
            p += 1
    assert False, "unreachable"
    
    return n, factors

In [104]:
# def factorize_list(lst):
#     result = {}
#     data = [n for n in range(range_number)]
#     for n in lst:
#         result[n] = factorize_naive(n)
#     return result

result = {}
workers = 16
with timer('Elapsed: {}s'):
    with multiprocessing.Pool(workers) as pool:
        input_data = (i for i in range(1, DATA_SIZE+1))
        result = pool.map(factorize_naive, input_data)
#       result = {
#             n:factors
#             for n, factors in pool.map(factorize_naive, input_data)
#         }
    
print(len(result), result[:100])

Elapsed: 5.704451322555542s
1000000 [[], [2], [3], [2, 2], [5], [2, 3.0], [7], [2, 2, 2], [3, 3], [2, 5.0], [11], [2, 2, 3.0], [13], [2, 7.0], [3, 5.0], [2, 2, 2, 2], [17], [2, 3, 3], [19], [2, 2, 5.0], [3, 7.0], [2, 11.0], [23], [2, 2, 2, 3.0], [5, 5], [2, 13.0], [3, 3, 3], [2, 2, 7.0], [29], [2, 3, 5.0], [31], [2, 2, 2, 2, 2], [3, 11.0], [2, 17.0], [5, 7.0], [2, 2, 3, 3], [37], [2, 19.0], [3, 13.0], [2, 2, 2, 5.0], [41], [2, 3, 7.0], [43], [2, 2, 11.0], [3, 3, 5.0], [2, 23.0], [47], [2, 2, 2, 2, 3.0], [7, 7], [2, 5, 5], [3, 17.0], [2, 2, 13.0], [53], [2, 3, 3, 3], [5, 11.0], [2, 2, 2, 7.0], [3, 19.0], [2, 29.0], [59], [2, 2, 3, 5.0], [61], [2, 31.0], [3, 3, 7.0], [2, 2, 2, 2, 2, 2], [5, 13.0], [2, 3, 11.0], [67], [2, 2, 17.0], [3, 23.0], [2, 5, 7.0], [71], [2, 2, 2, 3, 3], [73], [2, 37.0], [3, 5, 5], [2, 2, 19.0], [7, 11.0], [2, 3, 13.0], [79], [2, 2, 2, 2, 5.0], [3, 3, 3, 3], [2, 41.0], [83], [2, 2, 3, 7.0], [5, 17.0], [2, 43.0], [3, 29.0], [2, 2, 2, 11.0], [89], [2, 3, 3, 5.0], [7,

In [106]:
lst = []
workers = 16
with timer('Elapsed: {}s'):
    with multiprocessing.Pool(workers) as pool:
        input_data = (i for i in range(1, DATA_SIZE+1))
        result = pool.map(factorize_naive, input_data)
        
print(len(result), lst[:100])

Elapsed: 6.144156217575073s
1000000 [[], [2], [3], [2, 2], [5], [2, 3.0], [7], [2, 2, 2], [3, 3], [2, 5.0], [11], [2, 2, 3.0], [13], [2, 7.0], [3, 5.0], [2, 2, 2, 2], [17], [2, 3, 3], [19], [2, 2, 5.0], [3, 7.0], [2, 11.0], [23], [2, 2, 2, 3.0], [5, 5], [2, 13.0], [3, 3, 3], [2, 2, 7.0], [29], [2, 3, 5.0], [31], [2, 2, 2, 2, 2], [3, 11.0], [2, 17.0], [5, 7.0], [2, 2, 3, 3], [37], [2, 19.0], [3, 13.0], [2, 2, 2, 5.0], [41], [2, 3, 7.0], [43], [2, 2, 11.0], [3, 3, 5.0], [2, 23.0], [47], [2, 2, 2, 2, 3.0], [7, 7], [2, 5, 5], [3, 17.0], [2, 2, 13.0], [53], [2, 3, 3, 3], [5, 11.0], [2, 2, 2, 7.0], [3, 19.0], [2, 29.0], [59], [2, 2, 3, 5.0], [61], [2, 31.0], [3, 3, 7.0], [2, 2, 2, 2, 2, 2], [5, 13.0], [2, 3, 11.0], [67], [2, 2, 17.0], [3, 23.0], [2, 5, 7.0], [71], [2, 2, 2, 3, 3], [73], [2, 37.0], [3, 5, 5], [2, 2, 19.0], [7, 11.0], [2, 3, 13.0], [79], [2, 2, 2, 2, 5.0], [3, 3, 3, 3], [2, 41.0], [83], [2, 2, 3, 7.0], [5, 17.0], [2, 43.0], [3, 29.0], [2, 2, 2, 11.0], [89], [2, 3, 3, 5.0], [7,